# Load bigfish detection results


In [1]:
from dask.array.image import imread as imr
import os
import napari
import numpy as np
import pandas as pd
import trackpy as tp
import matplotlib.pyplot as plt
import bigfish
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack
import bigfish.plot as plot
from runBigfishDetection import saveSpotsNPZ, reorderZstack
from bigfish.detection.utils import get_object_radius_pixel
from buildReferenceSpot import buildReferenceSpotFromImages
from copy import deepcopy
pd.set_option('display.max_rows', 1500)
from runBigfishDetection import getSpotAndClusters, saveSpotsNPZ

In [2]:
homeFolder = '/media/rachel/9d56c1ff-e031-4e35-9f3c-fcd7d3e80033/Analysis/20230720/'
nameKey = 'Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_10ng.ml_tnf_exp1_4_F'
# homeFolder = '/home/rachel/single/spinningSequence/'
# nameKey = 'hela_K11_ON-_F'
imsQ = '11'
pathToTimeFrames = homeFolder+nameKey+imsQ+'/*.tif'
cellNumber = '1'
path_input = homeFolder+nameKey+imsQ+'/cell_'+str(cellNumber)+'/'
nucleiStackForm = nameKey+imsQ+"_cell_"
pathToTimeFramesCell = homeFolder+nameKey+imsQ+'/cell_'+str(cellNumber)+'/*.tif'
stackCell = imr(pathToTimeFramesCell)
MaxTimePoint = stackCell.shape[0]
path_input

'/media/rachel/9d56c1ff-e031-4e35-9f3c-fcd7d3e80033/Analysis/20230720/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_10ng.ml_tnf_exp1_4_F11/cell_1/'

In [3]:
stackCell = imr(pathToTimeFramesCell)
print(stackCell.shape)
maxImageCell = np.max(stackCell, axis=1)

(900, 13, 189, 189)


In [4]:
pathTocellCrops = homeFolder+nameKey+imsQ+'/cell_'+str(cellNumber)+'/'
moviePath = homeFolder+nameKey+imsQ

In [5]:
cellNumber = 1
spcl = np.load(pathTocellCrops+str(cellNumber)+'_spots_and_clusters.npz',allow_pickle=True)

spotsFrame = spcl['spotsFrame']
clustersFrames = spcl['clustersFrames']
ThresholdFrames = spcl['ThresholdFrames']
reference_spot = spcl['reference_spot']
refSpot = deepcopy(reference_spot)

In [6]:
def getTranscrtiptionSites(particle_1, MaxTimePoint, spotsFrame, clustersFrames, clusterID):
    potentialTxs = pd.DataFrame(np.zeros([MaxTimePoint, 1+3+1]), columns=['frame', 'z', 'y', 'x', 'mrna'])
    potentialTxs['frame']= np.arange(0,MaxTimePoint,1)
    

    for frameNumber in range(MaxTimePoint):
        t = particle_1.frame.values[np.argmin(abs(particle_1.frame.values-frameNumber))]
        particleCoordinates = np.asarray(particle_1[particle_1['frame']==t])[0][:2]
        coordinatesFound = spotsFrame[frameNumber][np.sum((spotsFrame[frameNumber][:,1:]-particleCoordinates)**2, axis=1)<=25,:]
        if coordinatesFound.size!=0:   
            if len(coordinatesFound) > 1:
                brightest = []
                for kk in range(len(coordinatesFound)):
                    brightest.append(np.array(maxImageCell[frameNumber][coordinatesFound[kk,1],coordinatesFound[kk,2]]))
                brightest = np.argmax(brightest)
                coordinatesFound = coordinatesFound[brightest].reshape((1,3))

            if len(coordinatesFound) ==1:
                potentialTxs.iloc[frameNumber,1:4] = coordinatesFound[0]
    # finding non empty frames

    txIndexes = np.where(np.sum(potentialTxs.iloc[:,1:4], axis=1)!=0)
    if np.size(txIndexes)!=0:
        txIndexes = txIndexes[0]
    
    # adding the mrna number

    for gg in txIndexes: 
        if np.sum(potentialTxs.iloc[gg,1:4])!=0:
            potentialTxs.iloc[gg,4]=1
        if clustersFrames[gg].size!=0:
            ixmin = np.argmin(np.sum((clustersFrames[gg][:,1:3] - np.array(potentialTxs.iloc[gg,2:4]))**2,axis=1))
            potentialTxs.iloc[gg,4]=clustersFrames[gg][ixmin,3]
    
    potentialTxs['cluster_id']=clusterID
    return potentialTxs

In [7]:
particle_1 = pd.read_pickle(moviePath+'/cellNumber_'+str(cellNumber)+'_particle_1.pkl') 
particle_1.index = np.arange(0,len(particle_1))
particle_1.track_length = len(particle_1)

txs1 = getTranscrtiptionSites(particle_1, MaxTimePoint, spotsFrame, clustersFrames, 0)

In [8]:
particle_2 = pd.read_pickle(moviePath+'/cellNumber_'+str(cellNumber)+'_particle_2.pkl') 
particle_2.index = np.arange(0,len(particle_2))
particle_2.track_length = len(particle_2)

txs2 = getTranscrtiptionSites(particle_2, MaxTimePoint, spotsFrame, clustersFrames, 1)

In [9]:
newClusterFrame = []
for hh in range(len(txs1)):
    if np.sum(txs1.iloc[hh,1:-1])==0:
        newClusterFrame.append(np.array([], dtype=np.int64).reshape((0,5)))
    else:
        newClusterFrame.append(np.array(txs1.iloc[hh,1:]).reshape((1,5)).astype(np.int64))
        
for hh in range(len(txs2)):
    if np.sum(txs2.iloc[hh,1:-1])==0:
        newClusterFrame[hh] = np.vstack([newClusterFrame[hh],np.array([], dtype=np.int64).reshape((0,5))])
    else:
        newClusterFrame[hh] = np.vstack([newClusterFrame[hh], np.array(txs2.iloc[hh,1:]).reshape((1,5)).astype(np.int64)])


In [10]:
moviePath

'/media/rachel/9d56c1ff-e031-4e35-9f3c-fcd7d3e80033/Analysis/20230720/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_10ng.ml_tnf_exp1_4_F11'

In [69]:
txs1.to_csv(moviePath+'/cellNumber_'+str(cellNumber)+'_particle_1_tracks.csv')
txs2.to_csv(moviePath+'/cellNumber_'+str(cellNumber)+'_particle_2_tracks.csv')

In [12]:
trackData = np.savez(moviePath+'/cellNumber_'+str(cellNumber)+'trackData',
                    tx1 = txs1,
                    tx2 = txs2,
                    newClusterFrame = newClusterFrame,
                    allow_pickle=True)

/home/rachel/anaconda3/envs/napari_env/lib/python3.9/site-packages/numpy/lib/npyio.py:696: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


In [65]:
def getDetectedPointsForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-1])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:]
    return pts_coords

def getDetectedClustersForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-3])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:3]
    return pts_coords

def set_pts_features(pts_layer, cls_layer, pts_coordinates, cluster_coordinate, step): #TxLayer
    # step is a 4D coordinate with the current slider position for each dim
    frameNumber = step[0]  # grab the leading ("time") coordinate
    pts_layer.data = getDetectedPointsForFrame(pts_coordinates,frameNumber)
    cls_layer.data = getDetectedClustersForFrame(cluster_coordinate,frameNumber)

pts_coordinates = spotsFrame
cluster_coordinate = newClusterFrame#clustersFrames
viewer = napari.Viewer()
image_layer = viewer.add_image(
        maxImageCell, colormap='green' #maxImageCell
        )
if image_layer.data.ndim == 4:
    bigfishSpots = spotsFrame
elif image_layer.data.ndim == 3:
    bigfishSpots = getDetectedPointsForFrame(pts_coordinates,int(np.shape(maxImageCell)[0]/2))
    
bigfish_Spots = viewer.add_points(
        getDetectedPointsForFrame(pts_coordinates,int(np.shape(maxImageCell)[0]/2)),
        face_color='#00000000',
        size=4,
        edge_width=0.3,
        edge_width_is_relative=False,
        edge_color='white',
        face_color_cycle = ['white'],
        name = 'bigFish Detected Spots'
        )

bigfish_clusters = viewer.add_points(
        getDetectedClustersForFrame(cluster_coordinate,int(np.shape(maxImageCell)[0]/2)),
        face_color='#00000000',
        size=8,
        edge_width=0.3,
        edge_width_is_relative=False,
        edge_color='red',
        face_color_cycle = ['red'],
        symbol='diamond',
        name = 'bigFish Clusters'
        )

# bigfish_TxSite = viewer.add_labels(Tx_label_clean, name='Tx Site',opacity=0.3)  bigfish_TxSite

viewer.dims.events.current_step.connect(
        lambda event: set_pts_features(bigfish_Spots, bigfish_clusters, pts_coordinates, cluster_coordinate, event.value)
        )

<function __main__.<lambda>(event)>

# Make feature table

In [1084]:
# lasthh=particle_1.frame[0]
# nextidx=0
# flag=0
# t=particle_1.frame[0]
# while (nextidx+1<particle_1.track_length[0] or lasthh<MaxTimePoint):
    
#     if np.asarray(particle_1[particle_1['frame']==lasthh]).size == 0:
#         while t<lasthh and nextidx+1<particle_1.track_length[0]:
#             nextidx = particle_1[particle_1['frame']==t].index[0]+1
#             if particle_1['frame'][nextidx]<= t+8:
#                 t=particle_1['frame'][nextidx]
#                 print('frame not found, searching in next frame :',t)
#             else:
#                 flag+=1       
#             if flag>1:
#                 nextidx = particle_1[particle_1['frame']==t].index[0]+1
#                 t=particle_1['frame'][nextidx]
#                 flag=0
#                 print('frame not found, searching in last frame closest to added frame :',t)
#     else:
#         if t!=lasthh:
#             t=lasthh
#             print('frame found, searching in frame :',t)
#         else:
#             nextidx = particle_1[particle_1['frame']==t].index[0]+1
#             lasthh = particle_1.frame[nextidx]
#             t = particle_1.frame[nextidx]
#             print('frame found, searching in frame :',t)
#             particleCoordinates = np.asarray(particle_1[particle_1['frame']==t])[0][:2]

#     addedPoints=0
#     for gg in range(9): 
#         hh = t+gg
#         if hh in particle_1.frame.values:
#             particleCoordinates = np.array(particle_1[particle_1['frame']==hh])[0][0:2]
#         if hh<900:
# #             print(hh, 'particleCoordinates = ',particleCoordinates)
#             coordinatesFound = spotsFrame[hh][np.sum((spotsFrame[hh][:,1:]-particleCoordinates)**2, axis=1)<=30,:]
#             if len(coordinatesFound) > 1:
#                 brightest = []
#                 for kk in range(len(coordinatesFound)):
#                     brightest.append(np.array(maxImageCell[hh][coordinatesFound[kk,1],coordinatesFound[kk,2]]))
#                 brightest = np.argmax(brightest)
#                 coordinatesFound = coordinatesFound[brightest].reshape((1,3))
# #                 print(hh,' ',coordinatesFound)
#             if len(coordinatesFound) ==1:
#                 potentialTxs.iloc[hh,1:4] = coordinatesFound[0]
# #                 print("New point added at ",hh)
#                 lasthh = hh
#                 addedPoints+=1
#         else:
#             lasthh=MaxTimePoint
#     if addedPoints==0:
#         print('no new points were added')
#         lasthh = t+1

#     print('lasthh = ', lasthh,', nextidx = ', nextidx, ', t=', t)